Fix the bug found in build_database_03.ipynb and redo the workflow. This time I adopt the pilot_test way of calculating composite depth, simply summing up the section start depth in the filename and section depth. In this case, I don't need to deal with core SO264-54-2 separately. The section depth can be found in the filename and in the conent. I prefer to read from the content because it has less chance to be accidently mess up by human. P.S. both chances are low actually.

Also, I adopt the bulk dataset from database_build_03.ipynb and merge it with the spe dataset produced here.

In [1]:
import numpy as np 
import pandas as pd
import glob
import matplotlib.pyplot as plt

#plt.style.use('ggplot')
plt.style.use('seaborn-colorblind')
plt.rcParams['figure.dpi'] = 300
plt.rcParams['savefig.dpi'] = 300
plt.rcParams['savefig.bbox'] = 'tight'

%matplotlib inline

import datetime
date = datetime.datetime.now().strftime('%Y%m%d')

# Read and build spectral datasets

## The original scans

In [50]:
core_list = glob.glob('data/SO264*')
core_list.sort()
core_list

['data/SO264-09-2',
 'data/SO264-13-2',
 'data/SO264-14-1',
 'data/SO264-15-2',
 'data/SO264-16-2',
 'data/SO264-19-2',
 'data/SO264-22-2',
 'data/SO264-24-3',
 'data/SO264-26-2',
 'data/SO264-28-2',
 'data/SO264-32-2',
 'data/SO264-34-2',
 'data/SO264-41-2',
 'data/SO264-44-2',
 'data/SO264-44-3',
 'data/SO264-45-2',
 'data/SO264-46-5',
 'data/SO264-47-2',
 'data/SO264-49-2',
 'data/SO264-51-2',
 'data/SO264-52-2',
 'data/SO264-53-2',
 'data/SO264-54-2',
 'data/SO264-55-1',
 'data/SO264-56-2',
 'data/SO264-60-12',
 'data/SO264-62-2',
 'data/SO264-64-1',
 'data/SO264-66-2',
 'data/SO264-70-1',
 'data/SO264-76-1']

The spe_dir list needs to be sorted to make sure the later duplicate replacing keeps the latest scanning only.

In [47]:
spe_dir[:5]

['data/SO264-55-1/Rescan_00050&01120/Run 1 at  10kV/SO264-55-1_00050  970.0mm  10s  10kV 150uA No-Filter   0.0mm Run1 Rep0 Sett Low.spe',
 'data/SO264-55-1/Rescan_00050&01120/Run 1 at  10kV/SO264-55-1_00050  650.0mm  10s  10kV 150uA No-Filter   0.0mm Run1 Rep0 Sett Low.spe',
 'data/SO264-55-1/Rescan_00050&01120/Run 1 at  10kV/SO264-55-1_00050  280.0mm  10s  10kV 150uA No-Filter   0.0mm Run1 Rep0 Sett Low.spe',
 'data/SO264-55-1/Rescan_00050&01120/Run 1 at  10kV/SO264-55-1_01120  980.0mm  10s  10kV 150uA No-Filter   0.0mm Run1 Rep0 Sett Low.spe',
 'data/SO264-55-1/Rescan_00050&01120/Run 1 at  10kV/SO264-55-1_01120  270.0mm  10s  10kV 150uA No-Filter   0.0mm Run1 Rep0 Sett Low.spe']

In [48]:
spe_dir.sort()
spe_dir[:5]

['data/SO264-55-1/Rescan_00050&01120/Run 1 at  10kV/SO264-55-1_00050   70.0mm  10s  10kV 150uA No-Filter   0.0mm Run1 Rep0 Sett Low.spe',
 'data/SO264-55-1/Rescan_00050&01120/Run 1 at  10kV/SO264-55-1_00050   80.0mm  10s  10kV 150uA No-Filter   0.0mm Run1 Rep0 Sett Low.spe',
 'data/SO264-55-1/Rescan_00050&01120/Run 1 at  10kV/SO264-55-1_00050   90.0mm  10s  10kV 150uA No-Filter   0.0mm Run1 Rep0 Sett Low.spe',
 'data/SO264-55-1/Rescan_00050&01120/Run 1 at  10kV/SO264-55-1_00050  100.0mm  10s  10kV 150uA No-Filter   0.0mm Run1 Rep0 Sett Low.spe',
 'data/SO264-55-1/Rescan_00050&01120/Run 1 at  10kV/SO264-55-1_00050  110.0mm  10s  10kV 150uA No-Filter   0.0mm Run1 Rep0 Sett Low.spe']

In [51]:
file_name = []
spe_all = []
depth_all = []
cps_all = []
core_all = []
s_depth_all = []

# do it core by core
for core in core_list:
    
    # only read the 10kV which having better signal to the light elements
    spe_dir = glob.glob('{}/Run 1 at  10kV/*.spe'.format(core))
    
    # make sure the order follows the depthes in filename
    spe_dir.sort()
    
    for spe in spe_dir:
        check_depth = spe.split()[3].split('_')[-1]
        
        # there are some inconsistencies in nameing...as usual
        # it's manual rescan in the original folder
        if check_depth == '10430B':
            start_depth = 10430
        # 5 and 6 digis means in mm
        elif len(check_depth) >= 5:
            start_depth = int(check_depth)
        # 3 and 4 digit means in cm, needs to be multipled to be mm
        elif len(check_depth) >= 3:
            start_depth = int(check_depth) * 10

        file_name.append(spe.split('/')[-1])
        
        with open(spe, 'r') as f:
            content = []
            lines = f.readlines()
            for line in lines[49:]:
                content = np.hstack((content, line.split()))
            
            section_depth = int(lines[13][:-3])
            spe_all.append(content.astype(int))
            cps_all.append(int(lines[28]))
            core_all.append(core[5:])
            s_depth_all.append(section_depth)
            depth_all.append(section_depth + start_depth)
            
    print('core {} is done.'.format(core))

core data/SO264-09-2 is done.
core data/SO264-13-2 is done.
core data/SO264-14-1 is done.
core data/SO264-15-2 is done.
core data/SO264-16-2 is done.
core data/SO264-19-2 is done.
core data/SO264-22-2 is done.
core data/SO264-24-3 is done.
core data/SO264-26-2 is done.
core data/SO264-28-2 is done.
core data/SO264-32-2 is done.
core data/SO264-34-2 is done.
core data/SO264-41-2 is done.
core data/SO264-44-2 is done.
core data/SO264-44-3 is done.
core data/SO264-45-2 is done.
core data/SO264-46-5 is done.
core data/SO264-47-2 is done.
core data/SO264-49-2 is done.
core data/SO264-51-2 is done.
core data/SO264-52-2 is done.
core data/SO264-53-2 is done.
core data/SO264-54-2 is done.
core data/SO264-55-1 is done.
core data/SO264-56-2 is done.
core data/SO264-60-12 is done.
core data/SO264-62-2 is done.
core data/SO264-64-1 is done.
core data/SO264-66-2 is done.
core data/SO264-70-1 is done.
core data/SO264-76-1 is done.


## The rescan

In [52]:
core_list = glob.glob('data/SO*/Rescan*')
core_list.sort()
core_list

['data/SO264-09-2/Rescan_0269&0369',
 'data/SO264-19-2/Rescan_03400',
 'data/SO264-28-2/Rescan_05755',
 'data/SO264-28-2/Rescan_06755',
 'data/SO264-51-2/Rescan_0000&0085',
 'data/SO264-53-2/Rescan_0000&0063&0163',
 'data/SO264-55-1/Rescan_00050&01120']

In [53]:
# do it core by core
for core in core_list:
    
    # only read the 10kV which having better signal to the light elements
    spe_dir = glob.glob('{}/Run 1 at  10kV/*.spe'.format(core))
    
    # make sure the order follows the depthes in filename
    spe_dir.sort()
    
    for spe in spe_dir:
        check_depth = spe.split()[3].split('/')[1].split('_')[1]

        # there are some inconsistencies in nameing...as usual
        # 5 and 6 digis means in mm
        if len(check_depth) >= 5:
            start_depth = int(check_depth)
        # 3 and 4 digit means in cm, needs to be multipled to be mm
        elif len(check_depth) >= 3:
            start_depth = int(check_depth) * 10

        file_name.append(spe.split('/')[-1])
        
        with open(spe, 'r') as f:
            content = []
            lines = f.readlines()
            for line in lines[49:]:
                content = np.hstack((content, line.split()))

            section_depth = int(lines[13][:-3])
            spe_all.append(content.astype(int))
            cps_all.append(int(lines[28]))
            core_all.append(core.split('/')[1])
            s_depth_all.append(section_depth)
            depth_all.append(section_depth + start_depth)

    print('core {} is done.'.format(core.split('/')[1]))   

core SO264-09-2 is done.
core SO264-19-2 is done.
core SO264-28-2 is done.
core SO264-28-2 is done.
core SO264-51-2 is done.
core SO264-53-2 is done.
core SO264-55-1 is done.


In [54]:
spe_df = pd.DataFrame(spe_all, columns = [str(_) for _ in range(2048)])
spe_df['cps'] = cps_all
spe_df['core'] = core_all
spe_df['composite_depth_mm'] = depth_all
spe_df['section_depth_mm'] = s_depth_all
spe_df['filename'] = file_name

In [55]:
spe_df.shape

(35330, 2053)

In [56]:
spe_df.head()

,0,1,2,3,4,5,6,7,8,9,...,2043,2044,2045,2046,2047,cps,core,composite_depth_mm,section_depth_mm,filename
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,174740,SO264-09-2,10,10,SO264-09-2_0000 10.0mm 10s 10kV 150uA No-F...
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,176896,SO264-09-2,20,20,SO264-09-2_0000 20.0mm 10s 10kV 150uA No-F...
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,183148,SO264-09-2,30,30,SO264-09-2_0000 30.0mm 10s 10kV 150uA No-F...
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,185151,SO264-09-2,40,40,SO264-09-2_0000 40.0mm 10s 10kV 150uA No-F...
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,186255,SO264-09-2,50,50,SO264-09-2_0000 50.0mm 10s 10kV 150uA No-F...


In [57]:
spe_df.tail()

,0,1,2,3,4,5,6,7,8,9,...,2043,2044,2045,2046,2047,cps,core,composite_depth_mm,section_depth_mm,filename
35325,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,93435,SO264-55-1,2080,960,SO264-55-1_01120 960.0mm 10s 10kV 150uA No-...
35326,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,93073,SO264-55-1,2090,970,SO264-55-1_01120 970.0mm 10s 10kV 150uA No-...
35327,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,92647,SO264-55-1,2100,980,SO264-55-1_01120 980.0mm 10s 10kV 150uA No-...
35328,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,91426,SO264-55-1,2110,990,SO264-55-1_01120 990.0mm 10s 10kV 150uA No-...
35329,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,69338,SO264-55-1,2120,1000,SO264-55-1_01120 1000.0mm 10s 10kV 150uA No-...


The shape is same as in build_database_02 (35330, 2053).

## Build composite_id

In [58]:
spe_df.composite_depth_mm.max()

19320

In [59]:
composite_id = []
for core, depth in zip(spe_df.core, spe_df.composite_depth_mm):
    composite_id.append('{}_{:05}'.format(core, depth))
    
spe_df['composite_id'] = composite_id

## Drop duplicates

In [60]:
clean_df = spe_df.drop_duplicates('composite_id', keep = 'last')
len(clean_df)

34322

### Check those duplicates

In [61]:
spe_df.loc[spe_df.composite_id.duplicated(keep = 'last'), spe_df.columns[-6:]].to_csv('results/check_duplicates_1.csv')
spe_df.loc[spe_df.composite_id.duplicated(keep = 'last'), spe_df.columns[-6:]]

,cps,core,composite_depth_mm,section_depth_mm,filename,composite_id
268,229565,SO264-09-2,2700,10,SO264-09-2_0269 10.0mm 10s 10kV 150uA No-F...,SO264-09-2_02700
269,252567,SO264-09-2,2710,20,SO264-09-2_0269 20.0mm 10s 10kV 150uA No-F...,SO264-09-2_02710
270,235857,SO264-09-2,2720,30,SO264-09-2_0269 30.0mm 10s 10kV 150uA No-F...,SO264-09-2_02720
271,246607,SO264-09-2,2730,40,SO264-09-2_0269 40.0mm 10s 10kV 150uA No-F...,SO264-09-2_02730
272,240093,SO264-09-2,2740,50,SO264-09-2_0269 50.0mm 10s 10kV 150uA No-F...,SO264-09-2_02740
...,...,...,...,...,...,...
34215,64206,SO264-76-1,3420,1160,SO264-76-1_0226 1160.0mm 10s 10kV 150uA No-F...,SO264-76-1_03420
34780,57196,SO264-51-2,880,880,SO264-51-2_0000_Rescan 880.0mm 10s 10kV 150...,SO264-51-2_00880
34944,19888,SO264-53-2,640,640,SO264-53-2_0000 640.0mm 10s 10kV 150uA No-F...,SO264-53-2_00640
34945,19318,SO264-53-2,650,650,SO264-53-2_0000 650.0mm 10s 10kV 150uA No-F...,SO264-53-2_00650


In [62]:
np.unique(spe_df.loc[spe_df.composite_id.duplicated(keep = 'last'), 'core'], return_counts = True)

(array(['SO264-09-2', 'SO264-13-2', 'SO264-14-1', 'SO264-15-2',
        'SO264-16-2', 'SO264-19-2', 'SO264-22-2', 'SO264-24-3',
        'SO264-26-2', 'SO264-28-2', 'SO264-34-2', 'SO264-41-2',
        'SO264-44-2', 'SO264-44-3', 'SO264-51-2', 'SO264-53-2',
        'SO264-54-2', 'SO264-55-1', 'SO264-56-2', 'SO264-60-12',
        'SO264-62-2', 'SO264-64-1', 'SO264-66-2', 'SO264-76-1'],
       dtype=object),
 array([200,   2,   1, 102,   2,   8,  11,   1,   6, 180,   4,   5,   1,
          1, 177, 260,   8,   2,   2,   1,   1,   1,   1,  31]))

The cores having more than 30 data pooints are weird. Let' check.

In [44]:
core_list

['data/SO264-09-2/Rescan_0269&0369',
 'data/SO264-19-2/Rescan_03400',
 'data/SO264-28-2/Rescan_05755',
 'data/SO264-28-2/Rescan_06755',
 'data/SO264-51-2/Rescan_0000&0085',
 'data/SO264-53-2/Rescan_0000&0063&0163',
 'data/SO264-55-1/Rescan_00050&01120']

Let's check those cores having more than 100 data points being duplicates.<br>
SO264-09-2, SO264-28-2, SO264-51-2, SO264-53-2: rescaned <br>
SO264-15-2: there are 0048 spes and 048 spes in the original folder, I reckon it's a rescan but not mentioned.<br>
SO264-76-1: most dupicates happen between the two scans 0004 and 0028 (starting from 4 cm and 28 cm respectively). The data from 4 cm are recognized as duplicates.<br>
The rest of the duplicates sparsely in each core are the bottom sanning which are exculded since there are scaning data in the same composite depth from the later section.

## Check nan

In [63]:
clean_df.isnull().values.any()

False

## Build section

In [64]:
clean_df = clean_df.set_index('composite_id')

In [68]:
clean_df[clean_df.section_depth_mm == 0]

,0,1,2,3,4,5,6,7,8,9,...,2043,2044,2045,2046,2047,cps,core,composite_depth_mm,section_depth_mm,filename
composite_id,,,,,,,,,,,,,,,,,,,,,


Interesting...no section depth start form 0 now.

In [69]:
clean_df.sort_values(by = 'composite_id', axis = 0, inplace = True)
clean_df

,0,1,2,3,4,5,6,7,8,9,...,2043,2044,2045,2046,2047,cps,core,composite_depth_mm,section_depth_mm,filename
composite_id,,,,,,,,,,,,,,,,,,,,,
SO264-09-2_00010,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,174740,SO264-09-2,10,10,SO264-09-2_0000 10.0mm 10s 10kV 150uA No-F...
SO264-09-2_00020,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,176896,SO264-09-2,20,20,SO264-09-2_0000 20.0mm 10s 10kV 150uA No-F...
SO264-09-2_00030,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,183148,SO264-09-2,30,30,SO264-09-2_0000 30.0mm 10s 10kV 150uA No-F...
SO264-09-2_00040,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,185151,SO264-09-2,40,40,SO264-09-2_0000 40.0mm 10s 10kV 150uA No-F...
SO264-09-2_00050,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,186255,SO264-09-2,50,50,SO264-09-2_0000 50.0mm 10s 10kV 150uA No-F...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SO264-76-1_04370,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,58656,SO264-76-1,4370,1120,SO264-76-1_0325 1120.0mm 10s 10kV 150uA No-F...
SO264-76-1_04380,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,54854,SO264-76-1,4380,1130,SO264-76-1_0325 1130.0mm 10s 10kV 150uA No-F...
SO264-76-1_04390,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,57354,SO264-76-1,4390,1140,SO264-76-1_0325 1140.0mm 10s 10kV 150uA No-F...


In [74]:
section_all = []
# make sure the order follows the core and composite depth
clean_df.sort_values(by = 'composite_id', axis = 0, inplace = True)

for core in np.unique(clean_df.core):    
    # I assume every core scanned from 0 mm so the first section in the core is marked as section 0
    # the deeper the larger number
    section = 0
    X = clean_df.loc[clean_df.core == core, 'section_depth_mm']
    for i in range(len(X)):
        section_all.append(section)
        try:
            # when section changes, the section depth should be rest to smaller number
            if X[i] > X[i + 1]:
                section += 1
        except IndexError:
            print('bottom of the core {}'.format(core))
        
clean_df['section'] = section_all

bottom of the core SO264-09-2
bottom of the core SO264-13-2
bottom of the core SO264-14-1
bottom of the core SO264-15-2
bottom of the core SO264-16-2
bottom of the core SO264-19-2
bottom of the core SO264-22-2
bottom of the core SO264-24-3
bottom of the core SO264-26-2
bottom of the core SO264-28-2
bottom of the core SO264-32-2
bottom of the core SO264-34-2
bottom of the core SO264-41-2
bottom of the core SO264-44-2
bottom of the core SO264-44-3
bottom of the core SO264-45-2
bottom of the core SO264-46-5
bottom of the core SO264-47-2
bottom of the core SO264-49-2
bottom of the core SO264-51-2
bottom of the core SO264-52-2
bottom of the core SO264-53-2
bottom of the core SO264-54-2
bottom of the core SO264-55-1
bottom of the core SO264-56-2
bottom of the core SO264-60-12
bottom of the core SO264-62-2
bottom of the core SO264-64-1
bottom of the core SO264-66-2
bottom of the core SO264-70-1
bottom of the core SO264-76-1


In [75]:
clean_df

,0,1,2,3,4,5,6,7,8,9,...,2044,2045,2046,2047,cps,core,composite_depth_mm,section_depth_mm,filename,section
composite_id,,,,,,,,,,,,,,,,,,,,,
SO264-09-2_00010,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,174740,SO264-09-2,10,10,SO264-09-2_0000 10.0mm 10s 10kV 150uA No-F...,0
SO264-09-2_00020,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,176896,SO264-09-2,20,20,SO264-09-2_0000 20.0mm 10s 10kV 150uA No-F...,0
SO264-09-2_00030,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,183148,SO264-09-2,30,30,SO264-09-2_0000 30.0mm 10s 10kV 150uA No-F...,0
SO264-09-2_00040,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,185151,SO264-09-2,40,40,SO264-09-2_0000 40.0mm 10s 10kV 150uA No-F...,0
SO264-09-2_00050,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,186255,SO264-09-2,50,50,SO264-09-2_0000 50.0mm 10s 10kV 150uA No-F...,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SO264-76-1_04370,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,58656,SO264-76-1,4370,1120,SO264-76-1_0325 1120.0mm 10s 10kV 150uA No-F...,3
SO264-76-1_04380,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,54854,SO264-76-1,4380,1130,SO264-76-1_0325 1130.0mm 10s 10kV 150uA No-F...,3
SO264-76-1_04390,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,57354,SO264-76-1,4390,1140,SO264-76-1_0325 1140.0mm 10s 10kV 150uA No-F...,3


In [76]:
clean_df[clean_df.core == 'SO264-76-1'].to_csv('results/check.csv')

It seems to be right...for now LOL.

## Export spe dataset

In [77]:
clean_df.to_csv('data/spe_dataset_{}.csv'.format(date))

# Merge spe and bulk datasets

In [78]:
bulk_df = pd.read_csv('data/bulk_dataset_20201007.csv', index_col = 0)
#spe_df = pd.read_csv('data/spe_dataset_20201008.csv', index_col = 0)

In [79]:
mask_c = spe_df.columns[:2048]  # only the channels
merge_df = pd.DataFrame()

for index, row in bulk_df.iterrows():
    mid = row['mid_depth_mm']
    core = row['core']
    
    # get the spe in 10 mm interval
    mask_r = (spe_df.composite_depth_mm >= (mid-5)) & (spe_df.composite_depth_mm <= (mid+5)) & (spe_df.core == core)
    merge_df = pd.concat(
        [merge_df, spe_df.loc[mask_r, mask_c].apply(np.mean, axis = 0).append(row)],
        axis = 1
    )
    
merge_df = merge_df.T.reset_index(drop = True)

In [80]:
merge_df

,0,1,2,3,4,5,6,7,8,9,...,2043,2044,2045,2046,2047,TC%,TOC%,CaCO3%,core,mid_depth_mm
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,2.54208,0.394127,17.8989,SO264-64-1,115
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,2.24715,0.611208,13.6323,SO264-64-1,215
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0.710588,0.523402,1.55982,SO264-64-1,305
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0.562171,0.472551,0.746802,SO264-64-1,1015
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0.578167,0.312852,2.21087,SO264-64-1,1815
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
383,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,6.22164,0.269061,49.6028,SO264-13-2,11435
384,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,7.96103,0.254319,64.2201,SO264-13-2,11535
385,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,6.59857,0.175505,53.5234,SO264-13-2,11835
386,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,8.53093,0.186714,69.5324,SO264-13-2,11935


In [81]:
merge_df[merge_df.isnull().any(axis = 1)]

,0,1,2,3,4,5,6,7,8,9,...,2043,2044,2045,2046,2047,TC%,TOC%,CaCO3%,core,mid_depth_mm
37,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.565117,0.385465,1.49704,SO264-66-2,395
149,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,9.91158,0.151331,81.3321,SO264-28-2,7595
150,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,7.79372,0.154413,63.6583,SO264-55-1,25
151,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,7.72836,0.131972,63.3007,SO264-55-1,75
330,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,8.79912,0.123676,72.2924,SO264-15-2,7805
331,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,10.9751,0.0985913,90.6339,SO264-15-2,8005


After checking the spe files, these are really data points not scanned by Avaatech.

In [88]:
clean_df = merge_df.dropna(axis = 0)
clean_df

,0,1,2,3,4,5,6,7,8,9,...,2043,2044,2045,2046,2047,TC%,TOC%,CaCO3%,core,mid_depth_mm
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,2.54208,0.394127,17.8989,SO264-64-1,115
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,2.24715,0.611208,13.6323,SO264-64-1,215
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0.710588,0.523402,1.55982,SO264-64-1,305
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0.562171,0.472551,0.746802,SO264-64-1,1015
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0.578167,0.312852,2.21087,SO264-64-1,1815
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
383,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,6.22164,0.269061,49.6028,SO264-13-2,11435
384,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,7.96103,0.254319,64.2201,SO264-13-2,11535
385,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,6.59857,0.175505,53.5234,SO264-13-2,11835
386,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,8.53093,0.186714,69.5324,SO264-13-2,11935


In [87]:
len(clean_df) + 6 - len(merge_df)

0

## Export dataset

In [89]:
clean_df.to_csv('data/spe+bulk_dataset_{}.csv'.format(date))